# Pandas確認課題

このPandas確認問題は、データサイエンス100本ノックの問題で最低限必要な問題を抜粋したものですが、[Introduction_to_Pandas](./11_Introduction_to_Pandas.ipynb) に掲載されていない機能が必要な問題もあります。
初めて触るライブラリを調べながら使うというのはよくある光景です。この課題では皆さんにもそれに挑戦していただきます。  
ヒントとして検索キーワードなどを載せておくので、自力で調べながら解いてみましょう。  



## 必要モジュールのインポート

この問題で使うモジュールをインポートします．

In [1]:
import pandas as pd
import numpy as np

## データの読み込み

In [2]:
df_customer = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/customer.csv')
df_product = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/product.csv')
df_receipt = pd.read_csv('https://raw.githubusercontent.com/The-Japan-DataScientist-Society/100knocks-preprocess/master/docker/work/data/receipt.csv')

---
## 問1. 条件抽出
> P-006: レシート明細データフレーム「df_receipt」から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [5]:
df_receipt.loc[(df_receipt['customer_id'] == 'CS018205000001') & ((df_receipt['amount'] >= 1000) | (df_receipt['quantity']>=5 )), ['sales_ymd', 'customer_id', 'product_cd', 'quantity', 'amount']]

,sales_ymd,customer_id,product_cd,quantity,amount
36,20180911,CS018205000001,P071401012,1,2200
9843,20180414,CS018205000001,P060104007,6,600
21110,20170614,CS018205000001,P050206001,5,990
68117,20190226,CS018205000001,P071401020,1,2200
72254,20180911,CS018205000001,P071401005,1,1100


---
## 問2. ソート
> P-18: 顧客データフレーム（df_customer）を生年月日（birth_day）で若い順にソートし、先頭5件を全項目表示せよ。

In [6]:
df_customer.sort_values('birth_day', ascending=False).head(5)

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
15639,CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
7468,CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
10745,CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
19811,CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
7039,CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1


---
## 問3. 全件数
> P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [7]:
len(df_receipt)

104681

## 問4. ユニーク件数
> P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [8]:
a=df_receipt['customer_id'].unique()
len(a)

8307

<details>
<summary>ヒント</summary>
「ユニーク」というのはそのまま検索に使える単語です。  
</details>

---
## 問5. 〇〇ごとに集計
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは先頭5件だけ表示せよ。

会員のみを抽出する方法は、例えば以下の2通りの方法があります。

In [4]:
df_receipt_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_receipt_only_member = df_receipt.query("not customer_id.str.startswith('Z')", engine="python")

In [26]:
total = df_receipt_only_member.groupby('customer_id')['amount'].sum()
average= total.mean()
total[total>average].head()

customer_id
CS001115000010    3044
CS001205000006    3337
CS001214000009    4685
CS001214000017    4132
CS001214000052    5639
Name: amount, dtype: int64

<details>
<summary>ヒント1</summary>
「pandas 要素ごと 集計」 などで今回使える機能に関する記事が見つかります。
</details>

<details>
<summary>ヒント2</summary>
メソッド名は "groupby" です。
</details>

---
## 問6. DataFrameの結合
> P-038: 顧客データフレーム（df_customer）とレシート明細データフレーム（df_receipt）から、各顧客ごとの売上金額合計を求めよ。ただし、買い物の実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが'Z'から始まるもの）は除外すること。なお、結果は先頭5件だけ表示せよ。

In [18]:
df_customer_only_member = df_customer[~df_customer["customer_id"].str.startswith("Z")]
df_customer_only_member = df_customer.query("not customer_id.str.startswith('Z')", engine="python")

In [34]:
df_customer_new = pd.merge(df_receipt, df_customer, on='customer_id', how='left')
df_customer_only_member = df_customer_new[(~df_customer_new["customer_id"].str.startswith("Z")) & (df_customer_new["gender_cd"]==1)]
df_customer_only_member.groupby('customer_id')['amount'].sum().reset_index
df_customer_only_member['amount'].fillna(0, inplace=True)
print(df_customer_only_member.head(5))



   sales_ymd  sales_epoch store_cd  receipt_no  receipt_sub_no   
0   20181103   1541203200   S14006         112               1  \
1   20181118   1542499200   S13008        1132               2   
2   20170712   1499817600   S14028        1102               1   
4   20180821   1534809600   S14025        1102               2   
5   20190605   1559692800   S13003        1112               1   

      customer_id  product_cd  quantity  amount customer_name  gender_cd   
0  CS006214000001  P070305012         1     158         志水 佳乃        1.0  \
1  CS008415000097  P070701017         1      81          中田 光        1.0   
2  CS028414000014  P060101005         1     170        米倉 ヒカル        1.0   
4  CS025415000050  P060102007         1      90         亀井 奈々        1.0   
5  CS003515000195  P050102002         1     138        梅村 真奈美        1.0   

  gender   birth_day   age postal_cd                   address   
0     女性  1996-12-08  22.0  224-0057   神奈川県横浜市都筑区川和町**********  \
1     女性  1971

C:\Users\kyuis\AppData\Local\Temp\ipykernel_13704\2621385232.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_customer_only_member['amount'].fillna(0, inplace=True)


<details>
<summary>ヒント1</summary>
タイトル通り 「pandas DataFrame 結合」などと調べれば必要な機能に関する記事が見つかります。  
</details>


<details>
<summary>ヒント2</summary>
"merge", "join"という似たメソッドがあります。  
今回の場合"merge"が便利でしょう。
</details>

---
## 問7. 時系列データ
> P-046: 顧客データフレーム（df_customer）の申し込み日（application_date）はYYYYMMD形式の文字列型でデータを保有している。これを日付型（dateやdatetime）に変換し、顧客ID（customer_id）とともに抽出せよ。なお、データは先頭5件を表示せよ。

In [3]:
import datetime
df_customer['date_year'] = pd.to_datetime(df_customer['application_date']).dt.date
df_customer.head()

,customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,date_year
0,CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,1970-01-01
1,CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,1970-01-01
2,CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,1970-01-01
3,CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0,1970-01-01
4,CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2,1970-01-01


<details>
<summary>ヒント1</summary>
「pandas datetime」などで該当の機能が見つかるかと思います。
</details>


<details>
<summary>ヒント2</summary>
"pd.to_datetime"というメソッドが使えるでしょう。このメソッドを適用する際ですが、for文を使わずに実装しましょう。

---
## 問8. 関数
> P-061: レシート明細データフレーム（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、合計した売上金額を常用対数化（底=10）して顧客ID、売上金額合計とともに表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、結果は先頭5件を表示せよ。

In [14]:
df_customer_only_member = df_receipt[~df_receipt["customer_id"].str.startswith("Z")]
df_customer_amount= df_customer_only_member.groupby('customer_id')['amount'].sum()
df_log_amount= np.log10(df_customer_amount)
df_result= pd.merge(df_customer_only_member, df_log_amount, on='customer_id', how='left')
df_result.rename(columns={'amount_x': 'original_value', 'amount_y': 'log_value' }, inplace=True)
df_result[['customer_id', 'original_value', 'log_value']].head()


,customer_id,original_value,log_value
0,CS006214000001,158,3.867114
1,CS008415000097,81,3.277609
2,CS028414000014,170,3.793930
3,CS025415000050,90,3.758609
4,CS003515000195,138,3.733358


---
## 問9. 欠損数
> P-079: 商品データフレーム（df_product）の各項目に対し、欠損数を確認せよ。

In [12]:
df_product.isnull().sum()

product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            7
unit_cost             7
dtype: int64

---
## 問10. 欠損値の除去
> P-080: 商品データフレーム（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たなdf_product_1を作成せよ。なお、削除前後の件数を表示させ、前設問で確認した件数だけ減少していることも確認すること。

In [18]:
before= len(df_product)

df_product_1= df_product.dropna()

after=len(df_product_1)

print(before)
print(after)




10030
10023


In [16]:
len(df_product), len(df_product_1)

(10030, 10023)

---
## 問11. 欠損値の穴埋め
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たなdf_product_2を作成せよ。なお、平均値について1円未満は四捨五入とせよ。補完実施後、各項目について欠損が生じていないことも確認すること。

In [14]:

mean_unit_price= round(df_product['unit_price'].mean())
df_product['unit_price'].fillna(mean_unit_price, inplace=True)
mean_unit_cost=  round(df_product['unit_cost'].mean())
df_product['unit_cost'].fillna(mean_unit_cost, inplace=True)

df_product_2= df_product.copy()

df_product_2.isnull().sum()








product_cd            0
category_major_cd     0
category_medium_cd    0
category_small_cd     0
unit_price            0
unit_cost             0
dtype: int64

### 余談
ChatGPTやBing AIに聞けば大抵のことは教えてくれます。  
何回か入力文章を吟味しないといけないこともありますが、知らないことを調べる場合は自分で検索するよりも早いです。  
ただ、ChatGPTなどは嘘をつく場合があるので、自分でソースを参照する姿勢は必要です。  

これはBingAIの回答例です。  

![BingAIの回答例](./imgs/pandas/BingAI.png)